In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
import rasterio
from matplotlib.patches import FancyArrowPatch, Polygon
from rasterio.plot import show


In [ ]:
country_colors = {
    "Tajikistan": "#8B5CF6",  # Purple
    "Kyrgyzstan": "#FBBF24",  # Amber Yellow
}

# Create figure and axes
fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={"projection": ccrs.PlateCarree()})

# Load and plot hillshade layers for both countries
# Replace these paths with your actual .tif file paths
kyrgyzstan_hillshade_path = "/Users/cooper/Desktop/CAMELS-CH/data/Hillshade/KGZ_hillshade.tif"
tajikistan_hillshade_path = "/Users/cooper/Desktop/CAMELS-CH/data/Hillshade/TJK_hillshade.tif"

# Open the hillshade files
with rasterio.open(kyrgyzstan_hillshade_path) as src1:
    # Plot the hillshade with appropriate styling
    show(src1, ax=ax, cmap="gray", alpha=0.5, transform=ccrs.PlateCarree())

with rasterio.open(tajikistan_hillshade_path) as src2:
    # Plot the hillshade with appropriate styling
    show(src2, ax=ax, cmap="gray", alpha=0.5, transform=ccrs.PlateCarree())

# Add cartography features
ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax.add_feature(cfeature.BORDERS, linewidth=0.5)
ax.add_feature(cfeature.LAND, alpha=0.0)
ax.add_feature(cfeature.OCEAN, alpha=0.0)
ax.add_feature(cfeature.LAKES, alpha=0.0)

# Plot CA basins with country-based coloring
for country, color in country_colors.items():
    country_basins = CA_gdf[CA_gdf["country"] == country]
    if not country_basins.empty:
        country_basins.plot(
            ax=ax,
            color=color,
            edgecolor="black",
            alpha=0.6,
            linewidth=0.5,
            label=country,
        )

# Add country labels for Central Asia
ax.text(
    74.7661,
    41.2044,
    "Kyrgyzstan",
    transform=ccrs.PlateCarree(),
    fontsize=10,
    fontweight="bold",
    ha="center",
)
ax.text(
    71.2761,
    38.8610,
    "Tajikistan",
    transform=ccrs.PlateCarree(),
    fontsize=10,
    fontweight="bold",
    ha="center",
)

# Add gridlines with labels
gl = ax.gridlines(draw_labels=True, linewidth=0.5, alpha=0.5, linestyle="--")
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {"size": 10}
gl.ylabel_style = {"size": 10}

for spine in ax.spines.values():
    spine.set_visible(False)

# Set specific spines to be visible for axes
ax.spines["left"].set_visible(True)  # y-axis
ax.spines["bottom"].set_visible(True)  # x-axis

arrow_x, arrow_y = 68.5, 42  # Position it in the top right
arrow_length = 0.5

# Create the arrow
north_arrow = FancyArrowPatch(
    (arrow_x, arrow_y),
    (arrow_x, arrow_y + arrow_length),
    transform=ccrs.PlateCarree(),
    color="black",
    linewidth=1,
    arrowstyle="-|>",
    mutation_scale=15,
)
ax.add_patch(north_arrow)

# Add "N" label
ax.text(
    arrow_x,
    arrow_y + arrow_length + 0.2,
    "N",
    transform=ccrs.PlateCarree(),
    ha="center",
    fontsize=10,
    fontweight="bold",
)

ax_inset = fig.add_axes([0.75, 0.1, 0.2, 0.2], projection=ccrs.Orthographic(72, 40), zorder=5)

# Add global features
ax_inset.stock_img()  # This adds the NASA Blue Marble image as a background

# Add coastlines and borders on top
ax_inset.add_feature(cfeature.COASTLINE, linewidth=0.3)
ax_inset.add_feature(cfeature.BORDERS, linewidth=0.2)

# Create a polygon to highlight the region shown in the main map
central_asia_extent = (67, 79, 37, 43)  # (min_lon, max_lon, min_lat, max_lat)
min_lon, max_lon, min_lat, max_lat = central_asia_extent

# Create the polygon vertices as a numpy array
poly_verts = np.array(
    [
        [min_lon, min_lat],
        [max_lon, min_lat],
        [max_lon, max_lat],
        [min_lon, max_lat],
        [min_lon, min_lat],
    ]
)

# Create and add the polygon
poly = Polygon(
    poly_verts,
    transform=ccrs.PlateCarree(),
    edgecolor="red",
    facecolor="red",
    alpha=1,
    linewidth=1.5,
)
ax_inset.add_patch(poly)

# Make the globe more complete by adjusting its appearance
ax_inset.set_global()  # Show the full globe

# Add a simple title to the inset map
ax_inset.set_title("Location", fontsize=12)

# Add a legend for countries
ax.legend(loc="lower right", fontsize=10)

plt.tight_layout()

plt.show()